In [ ]:
def calcular_densidade_grafo(file_path, num_vertices):
    """
    Questão 2a.
    Equipe: Eduardo Zirbell, Guilherme Kuhnen, Lucas Testoni
    
    Calcula a densidade de um grafo a partir de um arquivo contendo as arestas.
    
    Parâmetros:
        file_path (str): Caminho para o arquivo contendo as arestas do grafo.
        num_vertices (int): Número total de vértices no grafo.
        
    Retorna:
        float: Densidade do grafo.
    """
    # Contar o número de arestas no arquivo
    with open(file_path, 'r') as file:
        num_arestas = sum(1 for _ in file)
    
    # Fórmula da densidade
    densidade = (2 * num_arestas) / (num_vertices * (num_vertices - 1))
    return num_arestas, densidade

# Configurações
file_path = "data/scientometrics.net"
num_vertices = 1656

# Calcular densidade
num_arestas, densidade = calcular_densidade_grafo(file_path, num_vertices)

# Resultados
print(f"Número de arestas (M): {num_arestas}")
print(f"Densidade do grafo: {densidade:.4f}")

In [ ]:
def calcular_graus_medios(file_path, num_vertices):
    """
    
    Questão 2b.
    Equipe: Eduardo Zirbell, Guilherme Kuhnen, Lucas Testoni
    
    Calcula os graus médios de entrada e saída de um grafo a partir de um arquivo.
    
    Parâmetros:
        file_path (str): Caminho para o arquivo contendo as arestas do grafo.
        num_vertices (int): Número total de vértices no grafo.
        
    Retorna:
        tuple: (grau_medio_entrada, grau_medio_saida)
    """
    # Inicializar contadores
    num_arestas = 0

    # Processar o arquivo para contar arestas
    with open(file_path, 'r') as file:
        for line in file:
            num_arestas += 1  # Cada linha representa uma aresta
    
    # Grau médio de entrada e saída
    grau_medio_entrada = num_arestas / num_vertices
    grau_medio_saida = num_arestas / num_vertices

    return grau_medio_entrada, grau_medio_saida


# Configurações
file_path = "data/scientometrics.net"
num_vertices = 1656

# Calcular graus médios
grau_medio_entrada, grau_medio_saida = calcular_graus_medios(file_path, num_vertices)

# Resultados
print(f"Grau médio de entrada: {grau_medio_entrada:.2f}")
print(f"Grau médio de saída: {grau_medio_saida:.2f}")

In [ ]:
from collections import defaultdict

def ler_grafo(file_path):
    """
    Lê um arquivo contendo as arestas do grafo e cria uma representação de lista de adjacências.
    
    Parâmetros:
        file_path (str): Caminho para o arquivo contendo as arestas do grafo.
        
    Retorna:
        tuple: (grafo, grafo_transposto)
    """
    grafo = defaultdict(list)
    grafo_transposto = defaultdict(list)

    # Construir lista de adjacências e grafo transposto
    with open(file_path, 'r') as file:
        for line in file:
            u, v = map(int, line.split())  # u -> v
            grafo[u].append(v)
            grafo_transposto[v].append(u)

    return grafo, grafo_transposto


def kosaraju(grafo, grafo_transposto, num_vertices):
    """
    Implementa o algoritmo de Kosaraju para encontrar componentes fortemente conectados.
    
    Parâmetros:
        grafo (dict): Lista de adjacências do grafo.
        grafo_transposto (dict): Lista de adjacências do grafo transposto.
        num_vertices (int): Número de vértices no grafo.
        
    Retorna:
        int: Número de componentes fortemente conectados.
    """
    def dfs(v, visitado, stack, grafo_local):
        visitado[v] = True
        for vizinho in grafo_local[v]:
            if not visitado[vizinho]:
                dfs(vizinho, visitado, stack, grafo_local)
        stack.append(v)
    
    # Passo 1: Fazer a DFS para determinar a ordem de finalização dos vértices
    visitado = [False] * (num_vertices + 1)
    stack = []
    for i in range(1, num_vertices + 1):
        if not visitado[i]:
            dfs(i, visitado, stack, grafo)
    
    # Passo 2: Fazer a DFS no grafo transposto na ordem inversa
    visitado = [False] * (num_vertices + 1)
    num_componentes = 0
    while stack:
        v = stack.pop()
        if not visitado[v]:
            dfs(v, visitado, [], grafo_transposto)
            num_componentes += 1
    
    return num_componentes


# Configurações
file_path = "data/scientometrics.net"
num_vertices = 1656

# Construir o grafo e o grafo transposto
grafo, grafo_transposto = ler_grafo(file_path)

# Calcular o número de componentes fortemente conectados
num_componentes = kosaraju(grafo, grafo_transposto, num_vertices)

print(f"Número de componentes fortemente conectados: {num_componentes}")

In [ ]:
def analisar_grafo(file_path, num_vertices):
    """
    Análise de um grafo a partir de um arquivo contendo as arestas.
    
    Calcula:
        - Presença de ciclos no grafo.
        - Média do comprimento dos caminhos mais curtos entre os pares de vértices.
    
    Parâmetros:
        file_path (str): Caminho para o arquivo contendo as arestas do grafo.
        num_vertices (int): Número total de vértices no grafo.
        
    Retorna:
        dict: Resultados contendo:
            - "has_cycles" (bool): Indica se o grafo contém ciclos.
            - "average_shortest_path_length" (float ou None): 
                Média do comprimento dos caminhos mais curtos, ou None se não for possível calcular.
    """
    # Carregar o grafo como lista de adjacências
    graph = {i: [] for i in range(1, num_vertices + 1)}
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():
                source, target = map(int, line.split())
                graph[source].append(target)
    
    # Detectar ciclos (usando DFS)
    def has_cycle(graph):
        def dfs(vertex, visited, stack):
            visited.add(vertex)
            stack.add(vertex)
            for neighbor in graph[vertex]:
                if neighbor not in visited:
                    if dfs(neighbor, visited, stack):
                        return True
                elif neighbor in stack:
                    return True
            stack.remove(vertex)
            return False

        visited = set()
        for node in graph:
            if node not in visited:
                if dfs(node, visited, set()):
                    return True
        return False

    # Calcular caminhos mais curtos (usando BFS)
    def average_shortest_path(graph):
        def bfs(start):
            queue = [start]
            distances = {start: 0}
            while queue:
                current = queue.pop(0)
                for neighbor in graph[current]:
                    if neighbor not in distances:
                        distances[neighbor] = distances[current] + 1
                        queue.append(neighbor)
            return distances

        total_length = 0
        count = 0
        for vertex in graph:
            distances = bfs(vertex)
            total_length += sum(distances.values())
            count += len(distances) - 1  # Não contar o próprio nó como caminho

        return total_length / count if count > 0 else None

    # Analisar o grafo
    has_cycles = has_cycle(graph)
    avg_path_length = average_shortest_path(graph)

    return {
        "has_cycles": has_cycles,
        "average_shortest_path_length": avg_path_length
    }

# Configurações
file_path = "data/scientometrics.net"
num_vertices = 1656

# Analisar o grafo
resultados = analisar_grafo(file_path, num_vertices)

# Resultados
print(f"O grafo possui ciclos: {'Sim' if resultados['has_cycles'] else 'Não'}")
if resultados['average_shortest_path_length'] is not None:
    print(f"Média do comprimento dos caminhos mais curtos: {resultados['average_shortest_path_length']:.4f}")
else:
    print("Não foi possível calcular a média dos caminhos mais curtos.")

In [ ]:
def calcular_centralidade_grau_top3(file_path, num_vertices):
    """
    Calcula a centralidade de grau de cada vértice de um grafo direcionado
    e retorna os 3 principais vértices com base nos graus de entrada e saída.
    
    Parâmetros:
        file_path (str): Caminho para o arquivo contendo as arestas do grafo.
        num_vertices (int): Número total de vértices no grafo.
        
    Retorna:
        dict: Os 3 principais vértices por grau de entrada e saída.
    """
    # Inicializar lista de adjacências
    grau_entrada = {i: 0 for i in range(1, num_vertices + 1)}
    grau_saida = {i: 0 for i in range(1, num_vertices + 1)}
    
    # Construir o grafo a partir do arquivo
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():
                source, target = map(int, line.split())
                grau_saida[source] += 1
                grau_entrada[target] += 1

    # Ordenar vértices pelos graus de entrada e saída
    top3_grau_entrada = sorted(grau_entrada.items(), key=lambda x: x[1], reverse=True)[:3]
    top3_grau_saida = sorted(grau_saida.items(), key=lambda x: x[1], reverse=True)[:3]

    return {
        "top3_grau_entrada": top3_grau_entrada,
        "top3_grau_saida": top3_grau_saida
    }

# Configurações
file_path = "data/scientometrics.net"
num_vertices = 1656

# Calcular centralidade de grau
top3_centralidade = calcular_centralidade_grau_top3(file_path, num_vertices)

# Exibir resultados
print("Top 3 vértices por Grau de Entrada:")
for rank, (vertex, grau) in enumerate(top3_centralidade["top3_grau_entrada"], start=1):
    print(f"{rank}. Vértice {vertex} com Grau de Entrada = {grau}")

print("\nTop 3 vértices por Grau de Saída:")
for rank, (vertex, grau) in enumerate(top3_centralidade["top3_grau_saida"], start=1):
    print(f"{rank}. Vértice {vertex} com Grau de Saída = {grau}")